In [1]:
import torch
import numpy as np
import pickle

from common import Data, Split, Batches, load_data, encode_y, load_split
from utils import ProgressBar, evaluate
from pack import Pack
from model import Model, EncoderClassifierModel, TextPosterCombinedModel
from torch_models import TextPosterCombinedEncoder
from cls import BRClassifier
from train import train_epoches

%load_ext autoreload
%autoreload 2

In [2]:
genre_list = pickle.load(open("../data/tmdb_genres_list.pkl", 'rb'))

GENRES = load_data("../local/genres.pkl")
train = load_split("../local/train.pkl")
val = load_split("../local/val.pkl")
test = load_split("../local/test.pkl")
embedding = torch.load('../local/embedding.pth')
OVERVIEWS_ENCODED = load_data("../local/overviews_encoded.pkl")
TITLES_ENCODED = load_data("../local/titles_encoded.pkl")
POSTERS = load_data("../local/posters.npy")

In [3]:
text_encoder = torch.load("./saved/overview-lstm2/encoder_70_0.53.pth")
posters_encoder = torch.load("./saved/poster-res2/encoder_80_0.45.pth")

encoder = TextPosterCombinedEncoder(text_encoder, posters_encoder).cuda()

for param in encoder.parameters():
    param.requires_grad = False
    
classifier = BRClassifier(dims=[1024, 512], num_class=19, encoding_size=2048+1024, cuda=True)

optimizer = torch.optim.Adam(filter(lambda p:p.requires_grad, classifier.parameters()))
loss = torch.nn.BCEWithLogitsLoss().cuda()

scheduler=None

model = TextPosterCombinedModel(encoder, classifier, OVERVIEWS_ENCODED, POSTERS, GENRES)

In [4]:
n_epochs = 200 # change this to train
save_per_epoch = 1
for i in range(int(n_epochs/save_per_epoch)): 
    epoch_losses = train_epoches(n_epochs=save_per_epoch, model=model, train=train, loss=loss, val=val,
                  batch_size=32, optimizer=optimizer, scheduler=scheduler)
    bn = (i+1)*save_per_epoch
    torch.save(model.classifier, "./saved/cbn/cls_{}_{}.pth".format(bn, str(epoch_losses[1][1][-1])[:4]))

......................................5.....................................10......................................15.....................................20......................................25.....................................30......................................35.....................................40......................................45.....................................50......................................55.....................................60......................................65.....................................70......................................75.....................................80......................................85.....................................90......................................95.....................................100 F
epoch 0: 0.01578728
Val:	P(mi) 0.38330078125 	R(mi): 0.7245039224734656 	F1(mi): 0.5013571770716909

......................................5.....................................10............................

KeyboardInterrupt: 